In [233]:
import pandas as pd
import pickle

In [235]:
file_path = '../data/airoport_data.pkl'
with open(file_path, 'rb') as file:
    countries = pickle.load(file)

file_path = '../data/countries_continents.pkl'
with open(file_path, 'rb') as file:
    continents = pickle.load(file)

In [172]:
boarding = pd.read_csv('../data/BoardingData.csv', sep=';')

In [174]:
boarding

,PassengerFirstName,PassengerSecondName,PassengerLastName,PassengerSex,PassengerBirthDate,PassengerDocument,BookingCode,TicketNumber,Baggage,FlightDate,FlightTime,FlightNumber,CodeShare,Destination
0,SAVELII,VIKTOROVICH,RUSANOV,Male,03/10/1983,2879 096860,FRNINO,6625956945991971,Transit,2017-03-22,06:05,SU1369,Own,Moscow
1,LEV,MARKOVICH,ISAEV,Male,12/13/1975,1788 173211,Not presented,1643715499224676,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
2,NIKOLAI,I.,NIKOLSKII,Male,12/26/1990,4396 926588,VWNYGF,6247422701565929,Transit,2017-03-18,22:10,SU1180,Own,Volgograd
3,ANATOLII,PETROVICH,SHILOV,Male,05/24/1997,2595 919752,WQFFUE,Not presented,Registered,2017-03-18,22:10,SU1180,Own,Volgograd
4,MIROSLAVA,VIACHESLAVOVNA,SEMENOVA,Female,01/31/1976,6775 516990,Not presented,Not presented,Registered,2017-03-12,11:45,SU6284,Own,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155142,RAMIL,S.,DOROFEEV,Male,02/14/1994,3256 077151,SNRBXU,8653871661040452,Delayed,2017-05-18,06:30,SU1158,Own,Rostov
155143,RUSLAN,KONSTANTINOVICH,CHISTIAKOV,Male,02/14/1975,3497 458395,Not presented,0223584175158947,NaN,2017-05-18,06:30,SU1158,Own,Rostov
155144,SOFIA,VALEREVNA,KALMYKOVA,Female,12/17/1972,6618 352713,Not presented,Not presented,Registered,2017-10-02,07:30,SU5614,Own,Petropavlovsk
155145,VIACHESLAV,F.,KONONOV,Male,05/20/2000,4567 242088,Not presented,0975693658645383,Registered,2017-09-16,07:00,SU1640,Own,Omsk


### Отсутствует информация откуда совершаются полеты. Определим с помощью расписания

In [176]:
schedule = pd.read_csv('../data/SkyTeam_Timetable.csv')

### Оставим только нужные колонки

In [260]:
boarding['name'] = boarding['PassengerFirstName'] + ' ' + boarding['PassengerLastName']
boarding_df = boarding[['name', 'Baggage', 'FlightNumber', 'FlightTime', 'FlightDate']]
boarding_df = boarding_df.rename(columns={'FlightNumber':'Flight', 'FlightTime':'Dep_Time'})
boarding_df.head()

,name,Baggage,Flight,Dep_Time,FlightDate
0,SAVELII RUSANOV,Transit,SU1369,06:05,2017-03-22
1,LEV ISAEV,Registered,SU1180,22:10,2017-03-18
2,NIKOLAI NIKOLSKII,Transit,SU1180,22:10,2017-03-18
3,ANATOLII SHILOV,Registered,SU1180,22:10,2017-03-18
4,MIROSLAVA SEMENOVA,Registered,SU6284,11:45,2017-03-12


In [237]:
schedule_df = schedule[['From_Code', 'To_Code', 'Flight', 'Dep_Time', 'Arr_Time']]
schedule_df = schedule_df.drop_duplicates()
schedule_df

,From_Code,To_Code,Flight,Dep_Time,Arr_Time
0,CUL,MEX,AM169,06:00,08:55
1,MEX,CUL,AM162,06:50,07:52
3,MEX,CUL,AM162,06:55,08:01
4,CUL,MEX,AM169,06:01,08:50
5,MEX,CUL,AM162,07:00,08:06
...,...,...,...,...,...
1048076,PEK,JHG,MU5714,08:05,15:15
1048112,PEK,JNG,MU9704,19:30,21:15
1048113,JNG,PEK,MU9703,16:55,18:20
1048132,PEK,KHH,CI592,19:05,22:50


### Объединим таблицы по номеру полета

In [262]:
full_data = pd.merge(boarding_df, schedule_df, how='inner', on=['Flight', 'Dep_Time'])

In [264]:
full_data

,name,Baggage,Flight,Dep_Time,FlightDate,From_Code,To_Code,Arr_Time
0,SAVELII RUSANOV,Transit,SU1369,06:05,2017-03-22,STW,SVO,08:20
1,LEV ISAEV,Registered,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1
2,NIKOLAI NIKOLSKII,Transit,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1
3,ANATOLII SHILOV,Registered,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1
4,MIROSLAVA SEMENOVA,Registered,SU6284,11:45,2017-03-12,UUS,SVO,12:25
...,...,...,...,...,...,...,...,...
151254,IULIIA GORIUNOVA,Transit,SU1158,06:30,2017-05-18,SVO,ROV,08:35
151255,RAMIL DOROFEEV,Delayed,SU1158,06:30,2017-05-18,SVO,ROV,08:35
151256,RUSLAN CHISTIAKOV,NaN,SU1158,06:30,2017-05-18,SVO,ROV,08:35
151257,VIACHESLAV KONONOV,Registered,SU1640,07:00,2017-09-16,SVO,OMS,13:20


### Приведем имена колонок к удобному виду

In [266]:
full_data.columns = ['name', 'baggage', 'flight', 'dep_time', 'date', 'from_code', 'to_code', 'arr_time']

In [268]:
full_data.head()

,name,baggage,flight,dep_time,date,from_code,to_code,arr_time
0,SAVELII RUSANOV,Transit,SU1369,06:05,2017-03-22,STW,SVO,08:20
1,LEV ISAEV,Registered,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1
2,NIKOLAI NIKOLSKII,Transit,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1
3,ANATOLII SHILOV,Registered,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1
4,MIROSLAVA SEMENOVA,Registered,SU6284,11:45,2017-03-12,UUS,SVO,12:25


### Определим страны по коду аэропортов

In [270]:
full_data['departure_country'] = full_data['from_code'].map(countries)
full_data['arrival_country']   = full_data['to_code'].map(countries)

### Определим континенты по коду аэропортов

In [272]:
full_data['departure_continent'] = full_data['departure_country'].map(continents).fillna('Asia')
full_data['arrival_continent']   = full_data['arrival_country'].map(continents).fillna('Asia')

In [274]:
full_data.head()

,name,baggage,flight,dep_time,date,from_code,to_code,arr_time,departure_country,arrival_country,departure_continent,arrival_continent
0,SAVELII RUSANOV,Transit,SU1369,06:05,2017-03-22,STW,SVO,08:20,Russia,Russia,Europe/Asia,Europe/Asia
1,LEV ISAEV,Registered,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1,Russia,Russia,Europe/Asia,Europe/Asia
2,NIKOLAI NIKOLSKII,Transit,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1,Russia,Russia,Europe/Asia,Europe/Asia
3,ANATOLII SHILOV,Registered,SU1180,22:10,2017-03-18,SVO,VOG,01:05+1,Russia,Russia,Europe/Asia,Europe/Asia
4,MIROSLAVA SEMENOVA,Registered,SU6284,11:45,2017-03-12,UUS,SVO,12:25,Russia,Russia,Europe/Asia,Europe/Asia


### Создаем таблицу данных

In [276]:
def count_suspicious_flights(df):
    prev_airport = None
    susp_flights = cross_country = susp_cross_country = long_flights = 0
    for src_airport, dst_airport, src_country, dst_country, src_continent, dst_continent in df.values:
        if prev_airport != None and src_airport.lower() != prev_airport.lower():
            susp_flights += 1
            if src_country.lower() != dst_country.lower():
                susp_cross_country += 1
        prev_airport = dst_airport
        
        if src_country.lower() != dst_country.lower():
            cross_country += 1
        if src_continent.lower() != dst_continent.lower():
            long_flights += 1
    return susp_flights, cross_country, susp_cross_country, long_flights

In [288]:
suspicious_flights_df = pd.DataFrame(columns=['name', 'flights_count', 'suspicious_flights', 
                                              'cross_country', 'suspicious_cross_country', 'long_flights', 'baggage'])

for i, name in enumerate(full_data.name.unique()):
    df = full_data[full_data['name'] == name].sort_values(by='date').reset_index(drop=True)
    flights_count = len(df)
    baggage = full_data['baggage'][i]
    
    susp_flights, cross_country, susp_cross_country, long_flights = count_suspicious_flights(df[['from_code', 'to_code', 
                                                                                                 'departure_country', 'arrival_country', 
                                                                                                 'departure_continent', 'arrival_continent']])
    
    row_to_add = [name, flights_count, susp_flights, cross_country, susp_cross_country, long_flights, baggage]
    suspicious_flights_df.loc[i] = row_to_add
    


In [290]:
suspicious_flights_df

,name,flights_count,suspicious_flights,cross_country,suspicious_cross_country,long_flights,baggage
0,SAVELII RUSANOV,5,1,0,0,0,Transit
1,LEV ISAEV,5,3,0,0,0,Registered
2,NIKOLAI NIKOLSKII,9,5,0,0,0,Transit
3,ANATOLII SHILOV,3,1,0,0,0,Registered
4,MIROSLAVA SEMENOVA,4,0,0,0,0,Registered
...,...,...,...,...,...,...,...
49739,KRISTINA ROGOVA,1,0,0,0,0,Delayed
49740,NATALIIA GAVRILOVA,1,0,0,0,0,Delayed
49741,EMILIIA DIAKONOVA,1,0,0,0,0,NaN
49742,AMINA SUSLOVA,1,0,0,0,0,NaN


### Вычислим процент подозрительных и международных рейсов

In [295]:
suspicious_flights_df['percent_suspicious_flights'] = suspicious_flights_df['suspicious_flights'] / suspicious_flights_df['flights_count']

In [297]:
suspicious_flights_df['percent_cross_country'] = suspicious_flights_df['cross_country'] / suspicious_flights_df['flights_count']

In [299]:
suspicious_flights_df

,name,flights_count,suspicious_flights,cross_country,suspicious_cross_country,long_flights,baggage,percent_suspicious_flights,percent_cross_country
0,SAVELII RUSANOV,5,1,0,0,0,Transit,0.200000,0.0
1,LEV ISAEV,5,3,0,0,0,Registered,0.600000,0.0
2,NIKOLAI NIKOLSKII,9,5,0,0,0,Transit,0.555556,0.0
3,ANATOLII SHILOV,3,1,0,0,0,Registered,0.333333,0.0
4,MIROSLAVA SEMENOVA,4,0,0,0,0,Registered,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
49739,KRISTINA ROGOVA,1,0,0,0,0,Delayed,0.000000,0.0
49740,NATALIIA GAVRILOVA,1,0,0,0,0,Delayed,0.000000,0.0
49741,EMILIIA DIAKONOVA,1,0,0,0,0,NaN,0.000000,0.0
49742,AMINA SUSLOVA,1,0,0,0,0,NaN,0.000000,0.0


In [313]:
suspicious_flights_df.to_csv('../data/ready/BoardingData.csv')